In [1]:
# put this in it's own codeblock because it'll keep 
# running otherwise and eventually move to the root dir lol
%cd ../..

/Users/angus/code/jupyter/notebooks/20240907, vg


/Users/angus/code/jupyter/env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from lib.service.database import DatabaseService
from lib.defaults import INSTANCE_CFG
import os

os.makedirs('_out_img', exist_ok=True)

db = DatabaseService.create(INSTANCE_CFG[3].database, 1)
await db.wait_till_running()

dbname=au_land_db port=5434 user=postgres host=localhost password=throwAwayPassword


In [ ]:
from dataclasses import dataclass, field
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.colors import LogNorm, Normalize
from matplotlib import rc
import warnings

@dataclass
class Target:
    name: str
    figsize: tuple[int, int]
    figtext_y: tuple[float, float]
    leg_loc: str
    src_loc: str
    on_unzoned: bool = field(default=True)
    intersection: bool | None = field(default=None)

targets = [
    # Target('Sydney', (15, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    Target('Randwick', (15, 20), (0.125, 0.1), leg_loc="upper right", src_loc="lower left", intersection=0.8),
    # Target('Woollahra', (20, 18), (0.125, 0.1), leg_loc="lower right", src_loc="lower left"),
    # Target('Waverley', (20, 18), (0.125, 0.1), leg_loc="lower right", src_loc="lower left"),
    # Target('Burwood', (15, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left"),
    # Target('Strathfield', (15, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left"),
    # Target('The Hills', (10, 25), (0.125, 0.1), leg_loc="upper right", src_loc="lower left"),
    # Target('Parramatta', (22, 20), (0.2, 0.1), leg_loc="upper left", src_loc="lower left", intersection=0.8),
    # Target('Inner West', (22, 20), (0.2, 0.1), leg_loc="upper left", src_loc="lower left"),
    # Target('Canada Bay', (25, 20), (0.2, 0.1), leg_loc="upper left", src_loc="lower left"),
    # Target('Cumberland', (25, 15), (0.125, 0.1), leg_loc="upper right", src_loc="lower left"),
    # Target('Fairfield', (25, 15), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Liverpool', (35, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Canterbury-Bankstown', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Georges River', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Bayside (NSW)', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", intersection=0.8),
    # Target('Hunters Hill', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Ku-ring-gai', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Mosman', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('North Sydney', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Northern Beaches', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Ryde', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Sutherland', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False, intersection=0.8),
    # Target('Willoughby', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Hornsby', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
    # Target('Penrith', (22, 20), (0.125, 0.1), leg_loc="lower right", src_loc="lower left", on_unzoned=False),
]

rc('text', usetex=False)

ranked_zones = ['B1','B2','B3','B4','B5','B6','B7','E1','E2','E3','E4','E5',
                'IN1','IN2','MU1','R1','R2','R3','R4','R5','RU1','RU2','RU3','RU4','RU5','RU6',
                'SP1','SP3','SP4','SP5','W1','W3','W2','W4']
hidden_zones = ['C1', 'C2', 'C3', 'C4','SP2','RE1','RE2']

async def execute(query):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        async with db.async_connect() as conn:
            await conn.execute(query);
            await conn.commit();

for target in targets:
    await execute("DROP TABLE IF EXISTS lga_relevent")

    build_table_query = f"""
        CREATE TABLE lga_relevent AS
        SELECT property_id,
               pgeo.geometry as g,
               CASE 
                  WHEN zoning.zone_code IS NULL THEN {target.on_unzoned} 
                  WHEN zoning.zone_code NOT IN ('{"', '".join(hidden_zones)}') THEN TRUE
                  ELSE FALSE 
               END AS is_ranked,
               CASE WHEN zoning.zone_code IN ('{"', '".join(hidden_zones)}') THEN TRUE ELSE FALSE END AS is_hidden
           FROM abs.lga_2024 lga
           LEFT JOIN nsw_lrs.property_geometry pgeo ON ST_Intersects(pgeo.geometry, lga.geometry)
           LEFT JOIN (
                SELECT DISTINCT ON (property_id) *
                  FROM nsw_lrs.zone_observation z
                  ORDER BY property_id, effective_date DESC) zoning USING (property_id)
           WHERE lga_name ILIKE '{target.name}'
    """

    if target.intersection:
        build_table_query += f' AND (ST_Area(ST_Intersection(pgeo.geometry, lga.geometry)) / ST_Area(pgeo.geometry)) > {target.intersection}';
    await execute(build_table_query)

    lga_query = f"""
      SELECT geometry as g FROM abs.lga_2024 lga WHERE lga_name ILIKE '{target.name}'
    """
    
    unranked_query = f"""
      SELECT property_id, p.g as g
        FROM lga_relevent p
        WHERE NOT p.is_ranked AND NOT p.is_hidden
    """
            
    ranked_query = f"""
      WITH lv_sqm as (
        SELECT property_id, (lv.land_value / ST_Area(p.g::geography)) as lv, p.g as g
          FROM lga_relevent p
          LEFT JOIN (
            SELECT DISTINCT ON (property_id) *
              FROM nsw_vg.land_valuation lv
              ORDER BY property_id, effective_date DESC) lv USING (property_id)
          WHERE p.is_ranked AND lv.land_value IS NOT NULL)
        SELECT *, RANK() OVER (ORDER BY lv) AS rank  FROM lv_sqm
    """

    fig, ax = plt.subplots(figsize=target.figsize)

    lga_df = gpd.read_postgis(lga_query, db.engine(), geom_col='g')
    lga_df.plot(ax=ax, color='#cccccc', edgecolor="#cccccc")
    ranked_df = gpd.read_postgis(ranked_query, db.engine(), geom_col='g')
    ranked_df.plot(ax=ax, column='rank', cmap="magma")
    
    sm = plt.cm.ScalarMappable(cmap=plt.get_cmap("magma"), norm=Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, shrink=0.8, aspect=50, ticks=[0, 0.5, 1])
    cbar.set_label("Position within ordered Ranking", labelpad=0, fontsize=15)
    cbar.ax.tick_params(labelsize=15, rotation=90) 
    cbar.ax.set_yticklabels(["Lower", "", "Higher"])
    for label in cbar.ax.get_yticklabels():
        label.set_verticalalignment('center') 

    custom_legend = [Patch(facecolor='#cccccc', edgecolor="black", label=target.name)]
    ax.legend(handles=custom_legend, loc=target.leg_loc, title="Legend", fontsize=12, title_fontsize=10)
        
    plt.title("\nOrdered Ranking of Land\nBy SQM Land Value\nWithin '%s' LGA\n" % target.name, fontsize=20)
    source_box = AnchoredText(
        "Source: NSW Valuer General, published November 2024 accessed 2025",
        loc=target.src_loc,
        prop=dict(size=10),
        frameon=True,  # Draw a box around the text
    )
    source_box.patch.set_boxstyle("round,pad=0.1")
    source_box.patch.set_edgecolor("white")
    source_box.patch.set_facecolor("white")
    ax.add_artist(source_box)
    # plt.figtext(0.5, target.figtext_y[0], 
    #             "If you put the all properties here in a list and ordered of their SQM land " \
    #             "values, their position within this list would be their ranking. So keep in " \
    #             "mind this doesn't show the magnitude in the difference of price.",
    #             ha="center", fontsize=15, wrap=True, linespacing=1.5)
    plt.show()
    await execute("DROP TABLE IF EXISTS lga_relevent")
    
    